# Advance ingestion

To avoid manual ingestion, we provide higher level API to facilitate ingestion

In [ ]:
# ---------------- CREATE STORAGE SERVICE ----------------
def create_or_get_storage_service(om_server_con):
    # Check if the wanted storage service already exists, return the service entity
    service = om_server_con.get_by_name(entity="storageService", fqn=STORAGE_SERVICE_NAME)
    if service:
        return service

    custom_storage_conn = CustomStorageConnection(
        type="CustomStorage",
        sourcePythonClass="metadata.ingestion.source.storage.local.LocalStorageSource",
        connectionOptions={}
    )
    storage_conn_entity = StorageConnection(config=custom_storage_conn)

    service_request = CreateStorageServiceRequest(
        name=STORAGE_SERVICE_NAME,
        serviceType=StorageServiceType.CustomStorage,
        description="Local filesystem storage service",
        connection=storage_conn_entity
    )

    return om_server_con.create_or_update(data=service_request)

# ---------------- CREATE ROOT CONTAINER ----------------
def create_root_container(om_server_con, storage_service_entity):
    container = om_server_con.get_by_name(entity="container", fqn=f"{STORAGE_SERVICE_NAME}.{CONTAINER_NAME}")
    if container:
        return container

    container_request = CreateContainerRequest(
        name=CONTAINER_NAME,
        displayName="Root Container",
        service=storage_service_entity.fullyQualifiedName,
        dataModel=ContainerDataModel(isPartitioned=False)
    )
    return om_server_con.create_or_update(data=container_request)

# ---------------- CREATE FILE ENTITIES ----------------
# def register_files(om_server_con, storage_service_entity, container_entity, folder_path):
#     for root, dirs, files in os.walk(folder_path):
#         for f in files:
#             file_path = os.path.join(root, f)
#             size = os.path.getsize(file_path)
#             ext = os.path.splitext(f)[1]
#             mime = "text/csv" if ext == ".csv" else None  # simple heuristic
#             file_request = CreateFileRequest(
#                 name=f,
#                 displayName=f,
#                 service=storage_service_entity.fullyQualifiedName,
#                 directory=container_entity.fullyQualifiedName,
#                 fileType=FileType.CSV if ext==".csv" else FileType.Other,
#                 mimeType=mime,
#                 fileExtension=ext,
#                 path=file_path,
#                 size=size
#             )
#             om_server_con.create_or_update(data=file_request)
#             print(f"Registered file: {file_path}")